<a href="https://colab.research.google.com/github/JasonD28/CS-175/blob/master/model/mask_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [14]:
!pip install pydicom

     |████████████████████████████████| 35.5MB 18.8MB/s 


In [40]:
%%shell
# git clone https://github.com/JasonD28/CS-175.git
cd CS-175
cp model/dataset.py  ../

remote: Enumerating objects: 7, done.
remote: Counting objects: 100% (7/7), done.
remote: Compressing objects: 100% (1/1), done.
remote: Total 4 (delta 3), reused 4 (delta 3), pack-reused 0
Unpacking objects: 100% (4/4), done.
From https://github.com/JasonD28/CS-175
   eb5b381..46513cd  master     -> origin/master
Updating eb5b381..46513cd
Fast-forward
 model/dataset.py | 2 +-
 1 file changed, 1 insertion(+), 1 deletion(-)


In [0]:
import torch
from torch import optim
import torchvision
from dataset import PneumoniaDataset
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.models.detection.mask_rcnn import MaskRCNNPredictor

root = "drive/My Drive/cs-175-data"

In [0]:

model = torchvision.models.detection.maskrcnn_resnet50_fpn(pretrained=True)
in_features = model.roi_heads.box_predictor.cls_score.in_features
model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes=2)
in_features_mask = model.roi_heads.mask_predictor.conv5_mask.in_channels
hidden_layer = 256
model.roi_heads.mask_predictor = MaskRCNNPredictor(in_features_mask,
                                                   hidden_layer,
                                                   num_classes=2)

In [31]:
%%shell

# Using PyTorch helper functions to simplify training
# git clone https://github.com/pytorch/vision.git
cd vision
git checkout v0.5.1

cp references/detection/utils.py ../
cp references/detection/engine.py ../
cp references/detection/transforms.py ../
cp references/detection/coco_eval.py ../
cp references/detection/coco_utils.py ../

Already on 'v0.5.1'
Your branch is up to date with 'origin/v0.5.1'.


In [0]:
from engine import train_one_epoch, evaluate
import utils

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model.to(device)
dataset = PneumoniaDataset(root, True)
test_dataset = PneumoniaDataset(root, True)
indices = torch.randperm(len(dataset)).tolist()
dataset = torch.utils.data.Subset(dataset, indices[:-50])
test_dataset = torch.utils.data.Subset(test_dataset, indices[-50:])

data_loader = torch.utils.data.DataLoader(
        dataset, batch_size=2, shuffle=True, num_workers=0,
        collate_fn=utils.collate_fn)

test_data_loader = torch.utils.data.DataLoader(
    test_dataset, batch_size=1, shuffle=False, num_workers=0,
    collate_fn=utils.collate_fn)

optimizer = optim.Adam(model.parameters(), lr=1e-6)
lr_step = torch.optim.lr_scheduler.StepLR(optimizer, step_size=2, gamma=0.1)

# w = len(dataset)
# w
# data_loader = torch.utils.data.DataLoader(
#  dataset, batch_size=1, shuffle=True, num_workers=0)
#     # collate_fn=collate_fn)
# # For Training
# images,targets = next(iter(data_loader))
# images = list(image for image in images)
# idk = len(images)
# idk
# huh = len(targets)
# huh
# targets = [{k: v for k, v in t.items()} for t in [targets]]
# idk = len(targets)
# idk
# output = model(images,targets)   # Returns losses and detections
# # For inference
# model.eval()
# x = [torch.rand(1, 1024, 1024), torch.rand(1, 1024, 1024)]
# predictions = model(x)           # Returns predictions


In [53]:
for epoch in range(1):
    train_one_epoch(model, optimizer, data_loader, device, epoch, print_freq=10)
    lr_scheduler.step()
    evaluate(model, data_loader_test, device=device)


/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:2854: UserWarning: The default behavior for interpolate/upsample with float scale_factor will change in 1.6.0 to align with other frameworks/libraries, and use scale_factor directly, instead of relying on the computed output size. If you wish to keep the old behavior, please set recompute_scale_factor=True. See the documentation of nn.Upsample for details. 
  warnings.warn("The default behavior for interpolate/upsample with float scale_factor will change "


Loss is inf, stopping training
{'loss_classifier': tensor(0.5356, grad_fn=<NllLossBackward>), 'loss_box_reg': tensor(0., grad_fn=<DivBackward0>), 'loss_mask': tensor(0., grad_fn=<MulBackward0>), 'loss_objectness': tensor(18.0391, grad_fn=<BinaryCrossEntropyWithLogitsBackward>), 'loss_rpn_box_reg': tensor(inf, grad_fn=<DivBackward0>)}


SystemExit: ignored

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2890: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [54]:
%tb

SystemExit: ignored

In [0]:
|def train(model, optimizer, num_epochs = 1):
    for epoch in range(num_epochs):
        print('Starting epoch %d / %d' % (epoch + 1, num_epochs))
        model.train()
        for t, (x, y) in enumerate(loader_train):
            x_var = Variable(x.type(gpu_dtype))
            y_var = Variable(y.type(gpu_dtype).long())

            scores = model(x_var)
            
            if (t + 1) % print_every == 0:
                print('t = %d, loss = %.4f' % (t + 1, loss.item()))

            optimizer.zero_grad()
            optimizer.step()

In [0]:
train(model, )